In [1]:
import pyvista as pv
import numpy as np
import ufl
from dolfinx import mesh, fem, plot
from mpi4py import MPI
from petsc4py import PETSc

# Assurez-vous que votre maillage est correctement chargé
domain = xdmf_file.read_mesh(name="Grid")

# Définir les fonctions de forme et les fonctions test
V = fem.functionspace(domain, ("Lagrange", 1, (domain.geometry.dim, )))
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

# Définir la déformation
epsilon = ufl.sym(ufl.grad(u))

# Définir la contrainte
sigma = ufl.as_tensor([[sum(C_ijkl[i, j, k, l] * epsilon[k, l] for k in range(2) for l in range(2)) for j in range(2)] for i in range(2)])

T = fem.Constant(domain, default_scalar_type((0, 0)))
ds = ufl.Measure("ds", domain=domain)

# Paramètres pour la force centrifuge
rho = 1600       # Densité
Omega = 157      # Vitesse angulaire
hel = 1e-3       # Hauteur d'une couche
htot = N * hel
a = 0.5          # Largeur

# Volume of the section at x
x = ufl.SpatialCoordinate(domain)
V_section = htot * a * x[0]   # Volume of the section at x
Nx = rho * V_section * Omega**2

# Define the vector force expression
f = ufl.as_vector([Nx, 0])

# Définir la forme bilinéaire pour le problème de l'élasticité
a = ufl.inner(sigma, ufl.sym(ufl.grad(v))) * ufl.dx
L = ufl.dot(f, v) * ufl.dx + ufl.dot(T, v) * ufl.ds

# Définir le problème linéaire
uh = fem.Function(V)
problem = fem.petsc.LinearProblem(a, L, bcs=[bc], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()

# Pour les déplacements
u_topology, u_cell_types, u_geometry = plot.vtk_mesh(domain)
u_grid = pv.UnstructuredGrid(u_topology, u_cell_types, u_geometry)

# Adapter les vecteurs de déplacement en 3D pour PyVista
uh_vectors = np.zeros((u_geometry.shape[0], 3))
uh_vectors[:, :2] = uh.x.array.reshape(-1, 2)
u_grid.point_data["u"] = uh_vectors

# Calculer les déformations
u_array = uh.x.array.reshape(-1, 2)
def calculate_strain_displacement(u_array, mesh):
    nodes = mesh.geometry.x
    gradients = np.zeros((nodes.shape[0], 3))
    for i, node in enumerate(nodes):
        gradient = np.zeros((2, 2))
        # Remplir la matrice de déformation pour chaque noeud (simple exemple pour 2D)
        gradient[0, 0] = u_array[i, 0] / (node[0] + 1e-10)
        gradient[1, 1] = u_array[i, 1] / (node[1] + 1e-10)
        gradient[0, 1] = 0.5 * (u_array[i, 1] / (node[0] + 1e-10) + u_array[i, 0] / (node[1] + 1e-10))
        gradients[i, :2] = gradient.flatten()
    return gradients

deformation = calculate_strain_displacement(u_array, domain)
u_grid.point_data["deformation"] = np.linalg.norm(deformation, axis=1)

# Visualisation avec PyVista
p = pv.Plotter()
warped = u_grid.warp_by_scalar("u", factor=30)
p.add_mesh(warped, show_edges=True, scalar_bar_args={
    "title": "Déplacements",
    "title_font_size": 24,
    "label_font_size": 22,
    "shadow": True,
    "italic": True,
    "font_family": "arial",
    "vertical": False
})

# Ajouter la déformation
p.add_mesh(u_grid, scalars="deformation", cmap="coolwarm", show_edges=True, scalar_bar_args={
    "title": "Déformation",
    "title_font_size": 24,
    "label_font_size": 22,
    "shadow": True,
    "italic": True,
    "font_family": "arial",
    "vertical": False
})

p.add_text("Visualisation des déplacements et déformations", font_size=12, color="black", position="upper_edge")
p.show_bounds(color="black")
p.add_axes(color="black")
p.set_background("grey")
p.show()


NameError: name 'xdmf_file' is not defined